In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf


In [ ]:
## Reading the file from train and test csv files

test = pd.read_csv("test.csv")
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("train_old.csv")
train = pd.concat ([df1,df2], sort=False)



### Preprocessing and feature engineering ###

In [ ]:
# n_jobs was assigned to the max core available in the system i.e., 12 (otherwise for regression -1 will create issues.)

train.loc[train['n_jobs'] == -1, 'n_jobs'] = 12
test.loc[train_1['n_jobs'] == -1, 'n_jobs'] = 12

In [ ]:
## Feature Engineering. ##
# Creating 3 features after trying with different combinations using n_classes, n_classes_per_class, n_jobs and max_iter as all of them are interrelated.

train['max_samPerJob'] = (train['max_iter'] * train['n_samples'])/train['n_jobs']
train['n_clusPerJob'] = (train['n_classes'] * train['n_clusters_per_class'])/train['n_jobs']
train['clus_info'] = train['n_clusPerJob']/train['n_informative']

In [ ]:
test['max_samPerJob'] = (test['max_iter'] * test['n_samples'])/test['n_jobs']
test['n_clusPerJob'] = (test['n_classes'] * test['n_clusters_per_class'])/test['n_jobs']
test['clus_info'] = test['n_clusPerJob']/test['n_informative']

In [ ]:
y_time = train["time"]
pen_train = train["penalty"]
pen_test = test["penalty"]

In [ ]:
train = train.drop(['time'], axis=1)

In [ ]:
## Normalizing the data using standard scalar

num_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_num = df2.select_dtypes(include=num_dtypes)
test_num = test.select_dtypes(include=num_dtypes)

train_std = (train_num - test_num.mean())/test_num.std(ddof=0)

train_final = pd.concat([pen,train_std], axis=1)

train_final = train_final.drop(columns=['l1_ratio','scale','random_state','alpha','flip_y'])


In [ ]:
test_std = (test_num - test_num.mean())/test_num.std(ddof=0)
test_final = test_std.join(pen1)
test_final = test_final.drop(columns=['l1_ratio','scale','random_state','alpha','flip_y'])

### Training of the model using DNNLinearCombinedRegressor ###

In [ ]:
BATCH_SIZE = 150
num_epochs = 10000

X_train, X_test, y_train, y_test = train_test_split(train_final, y_time, test_size=0.3) 

input_train = tf.estimator.inputs.pandas_input_fn(x=train_final,y=y_time,batch_size=BATCH_SIZE,num_epochs=num_epochs,shuffle=True)

input_test = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=BATCH_SIZE,num_epochs=num_epochs,shuffle=True)

In [ ]:
n_classes = tf.feature_column.numeric_column("n_classes")
n_clusters_per_class = tf.feature_column.numeric_column("n_clusters_per_class")
n_informative = tf.feature_column.numeric_column("n_informative")
n1 = tf.feature_column.numeric_column("n_clusPerJob")
n2 = tf.feature_column.numeric_column("max_samPerJob")
n3 = tf.feature_column.numeric_column("clus_info")
max_iter = tf.feature_column.numeric_column("max_iter")
n_jobs = tf.feature_column.numeric_column("n_jobs")
n_samples = tf.feature_column.numeric_column("n_samples")
n_features = tf.feature_column.numeric_column("n_features")

penalty = tf.feature_column.categorical_column_with_vocabulary_list(key="penalty", vocabulary_list=["l2", "l1", "none", "elasticnet"])
pen = tf.feature_column.indicator_column(penalty)

In [ ]:

linear_feat_col = [    
    max_iter,
    n_jobs, 
    n_samples, 
    n_features, 
    n_classes, 
    n_clusters_per_class, 
    n_informative,
    n1,
    n2,
    n3,
]

DNN_feat_col = [
    max_iter,
    n_jobs, 
    n_samples, 
    n_features, 
    n_classes,
    n_clusters_per_class, 
    n_informative,
    n1,
    n2,
    n3,
    pen,
]


In [ ]:
model = tf.estimator.DNNLinearCombinedRegressor(
    linear_feature_columns=linear_feat_col,
    dnn_feature_columns=DNN_feat_col,
    dnn_hidden_units=[1500, 550, 360, 150, 75, 25, 14,7, 3],
    dnn_activation_fn=tf.nn.leaky_relu)
model.train(input_fn=input_train)


### Prediction on test data ###

In [ ]:
pred_inp_fn = tf.estimator.inputs.pandas_input_fn(
        x=test,
        batch_size=1,
        num_epochs=1,
        shuffle=False)

preds = m.predict(input_fn=pred_inp_fn)
res = []
for i in preds:
    res.append(i["predicted_label"][0])
print(res)

print(len(res))

c=0
for i in res:
    if i<0:
        c += 1
print(c)

In [ ]:
test_id = np.arange(100)
test_id = test_id.reshape(len(test_id),1)
res = np.array(res)
res = res.reshape(len(res),1)
res = np.abs(res)
out = np.concatenate((test_id,res), axis=1)
np.savetxt("Submission.csv", out, delimiter=",", fmt='%i,%f', header="Id,Time", comments='')